In [6]:
import torch
import torchvision.models as models
from torchvision import models,datasets,transforms
import os
import shutil
import pandas as pd
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

In [2]:
def copy_all_to_test(data_dir, output_dir):
    # Define class mapping for unifying class names
    class_mapping = {
        "cardboard": "cardboard",  
        "glass": "glass",
        "metal": "metal",
        "paper": "paper",
        "plastic": "plastic",
        "trash": "trash",
    }
 
    # Folders to ignore
    ignored_folders = {"Textile Trash", "Vegetation"}
 
    # Define test directory
    test_dir = os.path.join(output_dir, 'test')
    os.makedirs(test_dir, exist_ok=True)
 
    # Data storage for CSV logging
    records = []
 
    # Process each class directory
    for class_name in os.listdir(data_dir):
        if class_name in ignored_folders:
            print(f"Skipping {class_name} (ignored)")
            continue  # Skip ignored folders
 
        class_dir = os.path.join(data_dir, class_name)
        if not os.path.isdir(class_dir):
            continue
 
        # Map class name (default to original if no mapping exists)
        mapped_class = class_mapping.get(class_name, class_name)
 
        # Create mapped class directory in test set
        class_test_dir = os.path.join(test_dir, mapped_class)
        os.makedirs(class_test_dir, exist_ok=True)
 
        # Get all images
        images = [os.path.join(class_dir, img) for img in os.listdir(class_dir) if img.endswith(('png', 'jpg', 'jpeg'))]
 
        print(f"Copying {len(images)} images from {class_name} to {mapped_class} in test set...")
 
        # Copy all images to test set
        for image in images:
            dest = os.path.join(class_test_dir, os.path.basename(image))
            shutil.copy(image, dest)
            records.append((os.path.basename(image), class_name, mapped_class, "test"))  # Save mapping data
 
    # Save mapping information to CSV
    df = pd.DataFrame(records, columns=['filename', 'original_class', 'mapped_class', 'split_type'])
    df.to_csv(os.path.join(output_dir, 'class_mapping.csv'), index=False)
 
    print("All images successfully copied to the test set!")
 
# Set the paths
original_data_dir = '../data/dataset-resized'
output_data_dir = '../data/dataset_split'
 
# Copy all images to test set
copy_all_to_test(original_data_dir, output_data_dir)

Copying 403 images from cardboard to cardboard in test set...
Copying 501 images from glass to glass in test set...
Copying 410 images from metal to metal in test set...
Copying 594 images from paper to paper in test set...
Copying 482 images from plastic to plastic in test set...
Copying 137 images from trash to trash in test set...
All images successfully copied to the test set!


In [7]:
dir = '../data/dataset_split'

params = { 'batch_size':16,
           'shuffle':True,
           'num_workers':4 }

transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(256),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(os.path.join(dir, 'test'),transform = transform)

test_dataloader = torch.utils.data.DataLoader(test_dataset, **params)

class_names = test_dataset.classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
# Load the model architecture (must match the one used in training)
model = models.densenet121(pretrained=False)  # Or use the same architecture you used
num_classes = 6 # Ensure this matches the number of classes in your training dataset
model.classifier = torch.nn.Linear(model.classifier.in_features, num_classes)
# Load the pre-trained model weights
model_path = "save_model/trained_realwaste_diffusion_models_val_densenet.pth"
model.load_state_dict(torch.load(model_path))

# Set the model to evaluation mode
model.eval()

C:\Users\vidia\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\vidia\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [9]:
y_true = []  # True labels
y_pred = []  # Predicted labels

with torch.no_grad():  # Disable gradient computation for evaluation
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get class with highest probability

        # Store results
        y_true.extend(labels.cpu().numpy())   # Convert to numpy for evaluation
        y_pred.extend(predicted.cpu().numpy())

# Convert to NumPy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)


In [10]:
# Get class-wise accuracy
class_correct = np.zeros(num_classes)
class_total = np.zeros(num_classes)

for i in range(len(y_true)):
    label = y_true[i]
    if y_true[i] == y_pred[i]:
        class_correct[label] += 1
    class_total[label] += 1

# Print class-wise accuracy
for i in range(num_classes):
    if class_total[i] > 0:  # Avoid division by zero
        print(f'Accuracy of class {class_names[i]}: {100 * class_correct[i] / class_total[i]:.2f}%')


Accuracy of class cardboard: 80.40%
Accuracy of class glass: 64.27%
Accuracy of class metal: 44.63%
Accuracy of class paper: 41.75%
Accuracy of class plastic: 11.20%
Accuracy of class trash: 1.46%


In [11]:
# Generate classification report
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=class_names))

# Compute confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Classification Report:
               precision    recall  f1-score   support

   cardboard       0.27      0.80      0.40       403
       glass       0.62      0.64      0.63       501
       metal       0.55      0.45      0.49       410
       paper       0.63      0.42      0.50       594
     plastic       0.81      0.11      0.20       482
       trash       0.22      0.01      0.03       137

    accuracy                           0.45      2527
   macro avg       0.52      0.41      0.38      2527
weighted avg       0.57      0.45      0.43      2527

Confusion Matrix:
 [[324   0  21  53   2   3]
 [ 98 322  62  13   6   0]
 [176  27 183  19   3   2]
 [339   2   3 248   0   2]
 [179 167  55  27  54   0]
 [ 91   1   9  32   2   2]]


In [24]:
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_dataloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")

Test Accuracy: 49.31%
